# Imports

In [1]:
import pandas as pd
import pandasai as pai 
import os

from dotenv import load_dotenv

# pandasai imports
from pandasai_openai import OpenAI
from pandasai import SmartDataframe
from pandasai import clear_cache
from pandasai import Agent

# Initialization

In [2]:
READ_FROM_PICKLE = True

In [3]:
# finds .env file and loads the vars
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY", "Key not found")
openai_org = os.getenv("OPENAI_ORG", "Organization not found")

In [4]:
if READ_FROM_PICKLE:
    malicious_df = pd.read_pickle("data/malicious_features_numeric.pkl")
    benign_df = pd.read_pickle("data/benign_features_numeric.pkl")

ModuleNotFoundError: No module named 'numpy._core'

In [5]:
# Instantiate a LLM
llm = OpenAI(api_token=openai_api_key)

In [ ]:
malicious_smart = SmartDataframe(
    df=pd.DataFrame(malicious_df),
    config={"llm": llm, "verbose": True},
    name="Mirai botnet network packet capture.",
    description="A dataframe that is derived from a packet capture of the Mirai botnet network traffic.",
)

In [ ]:
benign_smart = SmartDataframe(
    df=pd.DataFrame(benign_df),
    config={"llm": llm, "verbose": True},
    name="Packet capture of normal operation network travvid.",
    description="A dataframe that is derived from a packet capture of the regular operation of a network.",
)

In [ ]:
malicious_smart.columns

# EDA

## Statistical

### Exploration via prompting

In [ ]:
top_5_source_IPs = malicious_smart.chat("Which are the 5 most popular source IP addresses?")

top_5_source_IPs

In [ ]:
print(malicious_smart.last_code_generated)

In [ ]:
top_5_dst_ports = malicious_smart.chat("Find the most used destination ports.")
top_5_dst_ports

In [ ]:
print(malicious_smart.last_code_generated)

### Prompt engineering

In [ ]:
rare_ports = malicious_smart.chat(
    "Which are the most commonly used known destination ports?"
)
rare_ports

In [ ]:
rare_ports = malicious_smart.chat(
    "Which are the most used destination ports less than or equal to 1024?"
)
rare_ports

### Correlation

In [ ]:
correlation = malicious_smart.chat(
    """1. Calculate the correlation between the source port and the length of a packet.
       2. Explain if the correlation that you calculated is significant and why.
    """
)
correlation

### Hypothesis testing

In [ ]:
hypothesis = malicious_smart.chat(
    "Is the difference between dst_ip_total_bytes and Packet Length statistically significant?"
)
hypothesis

### Outliers

In [ ]:
outliers = malicious_smart.chat(
    "Find the z score of the Packet Length and then calculate the top ten outliers."
)
outliers

## Visualizations

In [ ]:
malicious_smart.chat(
    "Plot the heatmap of the correlations of all variables."
)

In [ ]:
print(malicious_smart.last_code_executed)

In [ ]:
malicious_smart.chat("Plot a histogram for each of the variables.")

In [ ]:
print(malicious_smart.last_code_executed)

# Agents

In [ ]:
from pandasai import Agent

In [ ]:
agent = Agent(malicious_smart, config={"llm": llm},memory_size=1000)

In [ ]:
agent.chat('Is this dataframe indicating malicious or benign network behavior?')

In [ ]:
agent.chat('The dataframe that you have been given is a packet capture of computer network traffic. It has numerical features that characterize the packets that have been observed in this network. A packet capture is malicious if it has too many requests to ports that are unusual. Usual ports are 23 for Telnet and 80 for HTTP. Is this packet capture malicious or benign?')

In [ ]:
agent.explain()

# Clear cache

In [ ]:
clear_cache()